First I will check SIMC datasets. 
This EDA will follow steps in https://r4ds.hadley.nz/eda
First question is "What type of variation occurs within my variables?"
Before I answer this question, I'll look through datasets.
First thing to do is check whether all csv have same features. 

In [1]:
import os
import pandas as pd

In [2]:
csv_folder = r"C:\Users\sprin\Downloads\SIMC_OverlapTiffsWithPP\SIMC_OverlapTiffsWithPP\SIMC.Overlap.csv"

csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

first_csv = pd.read_csv(os.path.join(csv_folder, csv_files[0]))
reference_columns = set(first_csv.columns)

# Check all other CSVs
inconsistent_files = []
for csv_file in csv_files[1:]:
    df = pd.read_csv(os.path.join(csv_folder, csv_file))
    if set(df.columns) != reference_columns:
        inconsistent_files.append(csv_file)

if inconsistent_files:
    print("The following files have inconsistent columns:", inconsistent_files)
else:
    print("All CSV files have the same columns.")

The following files have inconsistent columns: ['20190503_SIMC_009_2mm_rep2_KG_data.csv', '20190506_SIMC_017_2mm_rep3_KG_data.csv', '20190508_SIMC_045_2mm_rep3_KG_data.csv', '20190508_SIMC_047_2mm_rep1_KG_data.csv', '20190514_SIMC_051_2mm_rep3_KG_data.csv', '20190514_SIMC_055_2mm_rep1_KG_data.csv', '20190515_SIMC_069_2mm_rep3_KG_data.csv', '20190523_SIMC_107_2mm_rep3_KG_data.csv', '20190527_SIMC_115_2mm_rep1_KG_data.csv']


Since we got some csvs having different features, we'll figure out what columns they have extra or missed.

In [3]:
column_sets = {}

for csv_file in csv_files:
    df = pd.read_csv(os.path.join(csv_folder, csv_file))
    column_sets[csv_file] = set(df.columns)

# Compare all CSV column sets
for file, columns in column_sets.items():
    if columns != reference_columns:
        missing = reference_columns - columns
        extra = columns - reference_columns
        print(f"\nFile: {file}")
        if missing:
            print(f"  Missing columns: {missing}")
        if extra:
            print(f"  Extra columns: {extra}")


File: 20190503_SIMC_009_2mm_rep2_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190506_SIMC_017_2mm_rep3_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190508_SIMC_045_2mm_rep3_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190508_SIMC_047_2mm_rep1_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190514_SIMC_051_2mm_rep3_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190514_SIMC_055_2mm_rep1_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190515_SIMC_069_2mm_rep3_KG_data.csv
  Extra columns: {'Biovolume..P..Spheroid.', 'Biovolume..Sphere.', 'Biovolume..Cylinder.'}

File: 20190523_SIMC_107_2mm_rep3_KG_data

Since there are some csv files containing extra three columns, and there are only 9 files out of 251 files, we'll drop features for EDA and further on. 
Now, I will check whether columns are in same order to combine csv files in one. 

In [4]:
order_mismatch_files = []

for csv_file in csv_files:
    df = pd.read_csv(os.path.join(csv_folder, csv_file))
    if list(df.columns) != reference_columns:
        order_mismatch_files.append(csv_file)

# Report if any files still have column order mismatches
if order_mismatch_files:
    print("The following files have column order mismatches:", order_mismatch_files)
else:
    print("All CSV files have the same column order.")

251